# PySpark DataFrame subsetting and cleaning

- After data inspection, it is often necessary to clean the data which mainly involves subsetting, renaming the columns, removing duplicated rows etc., PySpark DataFrame API provides several operators to do this. In this exercise, your job is to subset `'name'`, `'sex'` and `'date of birth'` columns from `people_df` DataFrame, remove any duplicate rows from that dataset and count the number of rows before and after duplicates removal step.

- Remember, you already have `SparkSession` `spark` and `people_df` DataFrames available in your workspace.

## Instructions

- Select `'name'`, `'sex'` and `'date of birth'` columns from `people_df` and create `people_df_sub` DataFrame.
- Print the first 10 observations in the `people_df` DataFrame.
- Remove duplicate entries from `people_df_sub` DataFrame and create `people_df_sub_nodup` DataFrame.
- How many rows are there before and after duplicates are removed?

In [2]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell''

In [3]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [4]:
file_path = "file:///home/talentum/test-jupyter/P3/M3/Dataset/people.csv"

# Create an DataFrame from file_path
people_df = spark.read.csv(file_path, header=True, inferSchema=True)

# Select name, sex and date of birth columns
people_df_sub = people_df.select('name', 'sex', 'date of birth')

# Print the first 10 observations from people_df_sub
people_df_sub.show(10)

# Remove duplicate entries from people_df_sub
people_df_sub_nodup = people_df_sub.dropDuplicates()

# Count the number of rows
print("There were {} rows before removing duplicates, and {} rows after removing duplicates".format(people_df_sub.count(), people_df_sub_nodup.count()))

+----------------+------+-------------+
|            name|   sex|date of birth|
+----------------+------+-------------+
|  Penelope Lewis|female|   1990-08-31|
|   David Anthony|  male|   1971-10-14|
|       Ida Shipp|female|   1962-05-24|
|    Joanna Moore|female|   2017-03-10|
|  Lisandra Ortiz|female|   2020-08-05|
|   David Simmons|  male|   1999-12-30|
|   Edward Hudson|  male|   1983-05-09|
|    Albert Jones|  male|   1990-09-13|
|Leonard Cavender|  male|   1958-08-08|
|  Everett Vadala|  male|   2005-05-24|
+----------------+------+-------------+
only showing top 10 rows

There were 100000 rows before removing duplicates, and 99998 rows after removing duplicates


In [8]:
file_path = "file:///home/talentum/test-jupyter/P3/M3/Dataset/people.csv"

# Create an DataFrame from file_path
people_df = spark.read.csv(file_path, header=True, inferSchema=True)

# Select name, sex and date of birth columns
people_df_sub = people_df.select('name', 'sex', 'date of birth')

people_df_sub_count= people_df_sub.groupby('name', 'sex', 'date of birth').count()

people_df_sub_duplicate=people_df_sub_count.filter('count>1')
people_df_sub_duplicate.show()

+-------------+------+-------------+-----+
|         name|   sex|date of birth|count|
+-------------+------+-------------+-----+
|Kathryn Davis|female|  20175-02-28|    2|
| Robert Smith|  male|  20175-02-28|    2|
+-------------+------+-------------+-----+



In [17]:
people_df.groupby(people_df.name, people_df.sex, people_df['date of birth']).count().filter('count>1').show()

+-------------+------+-------------+-----+
|         name|   sex|date of birth|count|
+-------------+------+-------------+-----+
|Kathryn Davis|female|  20175-02-28|    2|
| Robert Smith|  male|  20175-02-28|    2|
+-------------+------+-------------+-----+



In [18]:
people_df\
.groupby(people_df.name, people_df.sex, people_df['date of birth'])\
.count()\
.filter('count>1')\
.show()

+-------------+------+-------------+-----+
|         name|   sex|date of birth|count|
+-------------+------+-------------+-----+
|Kathryn Davis|female|  20175-02-28|    2|
| Robert Smith|  male|  20175-02-28|    2|
+-------------+------+-------------+-----+



In [20]:
import pyspark.sql.functions as F

In [21]:
people_df\
.groupby(people_df.name, people_df.sex, people_df['date of birth'])\
.count()\
.filter(F.col('count')>1)\
.show()

+-------------+------+-------------+-----+
|         name|   sex|date of birth|count|
+-------------+------+-------------+-----+
|Kathryn Davis|female|  20175-02-28|    2|
| Robert Smith|  male|  20175-02-28|    2|
+-------------+------+-------------+-----+



In [23]:
print(type(F.col('count')>1))

<class 'pyspark.sql.column.Column'>


In [26]:
people_df_sub_count.exceptAll(people_df_sub_duplicate).show()

+-----------------+------+-------------+-----+
|             name|   sex|date of birth|count|
+-----------------+------+-------------+-----+
|   Peggy Weinberg|female|   1983-07-28|    1|
| Christina Bolduc|female|   1982-09-02|    1|
|  Margaret Mackie|female|   1965-12-30|    1|
|Richard Constance|  null|   1983-06-23|    1|
|      Iris Castro|female|   1993-09-08|    1|
|  Maxine Dettloff|female|   1983-06-22|    1|
| Gregory Ferguson|  male|   1982-06-07|    1|
|      Elvis Stein|  male|   2014-07-29|    1|
|      Mary Dunbar|female|   1954-09-24|    1|
|     Ruth Dickson|female|   1983-02-03|    1|
|     Wendy Roybal|female|   1999-03-22|    1|
| Jeannie Kappeler|female|   1965-01-29|    1|
|      Nancy Evans|female|   1967-05-13|    1|
|   Michael Ferrer|  male|   1945-12-29|    1|
|     Dewey Torres|  male|   1978-12-13|    1|
| Willie Sanderson|  male|   1968-06-27|    1|
|      Diane Lopes|female|   1970-09-04|    1|
|     Tim Angelico|  male|   2014-10-08|    1|
|  Heather Si